# Frequent pattern mining

## Import data and libraries

In [ ]:
# import sys
# import os
# import scipy as sc
# from scipy import stats
import numpy as np
# import pandas as pd
# import datetime as dt
import math
import pickle
# import scipy.cluster.hierarchy as shc
from tqdm import tqdm
# import time
from spmf import Spmf
import copy



In [ ]:
# File export suffix
start_index = 0
number_of_stays = 100
display_matrix = True
output_path = "output/"
run_test_data = True
output_folder = f"stays-{number_of_stays}/"
cluster_level = 0.725
# cluster_level = 1
# output_folder = f"stays-test-{number_of_stays}/"

# file_suffix = '_test_' + str(number_of_stays)
file_suffix = '_' + str(number_of_stays)


### Store and load function


In [ ]:
def save_as_pickle(data, file_name, path=output_path):
    file = open(path + file_name, 'wb')
    pickle.dump(data, file)
    file.close()


def get_pickle(file_name, path=output_path):
    return pickle.load(open(path + file_name, 'rb'))



In [ ]:
clusters = get_pickle('alignments' + file_suffix)


## Frequent pattern mining

In [ ]:
def get_clustered_events(level):
    keys = [float(cluster) for cluster in clusters.keys()]
    keys.pop(0)

    cluster_level = 0
    level = float(level)

    # TODO possibly change this to a binary search variant --> O(n) to O(log(n))
    for clustered_level in keys:

        if (level >= cluster_level and level < clustered_level):
            break

        else:
            cluster_level = clustered_level

    # return clusters[cluster_level]
    return copy.deepcopy(clusters[cluster_level])


In [ ]:
clusters.keys()


### Fetch sequences at level


In [ ]:
# c = [seq['sequence'] for seq in get_clustered_events(0)]
c = [seq['sequence'] for seq in get_clustered_events(cluster_level)]
# c = c1.copy()
# c = c1

len(c)


In [ ]:
c[0]

### Replace all events with character

In [ ]:
def sequence_to_list_of_strings(c):
    transformed_sequences = copy.deepcopy(c)

    for c_index, cluster in enumerate(transformed_sequences):
        # print(cluster)
        for e_index, event in enumerate(cluster):
            # print(f"event: {event}, instance: {isinstance(event, list)}")

            if isinstance(event, list):
                for index, e in enumerate(event):
                    # print(f"e: {e}")
                    if e == '-':
                        transformed_sequences[c_index][e_index][index] = 'x'
                        # c[c_index][e_index][index] = 100
                    else:
                        transformed_sequences[c_index][e_index][index] = e

                s = transformed_sequences[c_index][e_index]
                s.sort()
                transformed_sequences[c_index][e_index] = "".join(s)
            # elif len(event) == 1:
            else:
                # c[c_index][e_index] = int(event)
                transformed_sequences[c_index][e_index] = event

        transformed_sequences[c_index] = " ".join(
            transformed_sequences[c_index])

    return transformed_sequences


# print(c)


In [ ]:
# c = sequence_to_list_of_strings(c)
seq = copy.deepcopy(c)

## VGEN

In [ ]:
def get_frequent_patterns(input=[], min_sup=0.1, max_gap='1', max_pat_length=""):
    spmf = Spmf("VGEN", input_direct=input,
                input_type="text",
                output_filename="output.txt", spmf_bin_location_dir="/Users/youri/Downloads",
                arguments=[min_sup, max_pat_length, str(max_gap), True])
    spmf.run()
    # print(spmf.parse_output())
    return spmf.to_pandas_dataframe()


### Optional: lengthen the dataset artificially

In [ ]:
# c1 = c.copy()
# c1 = c
# for i in range(5):
#     c1 = c1 + c
# print(f"c: {len(c)}, c1: {len(c1)}")
for i in c:
    print(f"length seq: {len(i)}")
    # print(i[0:10])
    # print()
    
# 17416


### Mine patterns

In [ ]:
def replace_gaps_and_to_string(c):
    seqs = copy.deepcopy(c)
    for s_index, sequence in enumerate(seqs):
        for e_index, event in enumerate(sequence):
            if isinstance(event, list):
                s_temp = [e if e != "-" else 'x' for e in event]
                s_temp.sort()
                seqs[s_index][e_index] = "".join(s_temp)

        seqs[s_index] = " ".join(seqs[s_index])
    return seqs
    


In [ ]:
SEGMENT_SIZE = 4000
FIRST_PERCENTILE = 2183
FIRST_PERCENTILE = 8000
segmented_sequences = []
sequence_to_split_pattern_lookup = []
sequences = copy.deepcopy(c)


# TODO correct support

# Split sequences to segmented sequences if larger than first percentile, else just add whole sequence
for s_index, sequence in enumerate(sequences):
    # print(type(sequence))
    # print(sequence[0:100])
    if len(sequence) > FIRST_PERCENTILE:
        print(
            f"Sequence too long: {len(sequence)} - splitting: {math.ceil(len(sequence) / SEGMENT_SIZE)} x {SEGMENT_SIZE} = {math.ceil(len(sequence) / SEGMENT_SIZE) * SEGMENT_SIZE}")
        for i in range(0, math.ceil(len(sequence) / SEGMENT_SIZE)):
            segmented_sequences.append(
                sequence[i * SEGMENT_SIZE: (i + 1) * SEGMENT_SIZE - 1])
        sequence_to_split_pattern_lookup.append(s_index +
                                        math.ceil(len(sequence) / SEGMENT_SIZE) - 1)
    else:
        segmented_sequences.append(sequence)
        sequence_to_split_pattern_lookup.append(s_index)
        
# Transform (segmented) list sequences to (segmented) string sequences
sequences = replace_gaps_and_to_string(segmented_sequences)

# for s in sequences:
#     print(s[0:100])
#     print()
    
print("-- Finished preparation --")

patterns = get_frequent_patterns(sequences, max_gap=1)
# patterns = get_frequent_patterns(c1, max_gap=1)
print(f"\n#patterns found: {len(patterns)}")
print("\n-- Done --")


In [ ]:
# LENGTH_THRESHOLD = 20000
# sequences = []
# sequence_to_split_pattern_lookup = []

# # for index, sequence in enumerate(c1):
# for index, sequence in enumerate(c):

#     if len(sequence) > LENGTH_THRESHOLD:
#         # print(f"Index if: {index}")
#         print(
#             f"Sequence too long: {len(sequence)} - splitting: {math.ceil(len(sequence) / LENGTH_THRESHOLD)} x {LENGTH_THRESHOLD} = {math.ceil(len(sequence) / LENGTH_THRESHOLD) * LENGTH_THRESHOLD}")
#         for i in range(0, math.ceil(len(sequence) / LENGTH_THRESHOLD)):
#             # print(f"Part {i}: {i * LENGTH_THRESHOLD}-{(i + 1)* LENGTH_THRESHOLD - 1}")
#             partial_sequence = sequence[i* LENGTH_THRESHOLD: (i + 1) * LENGTH_THRESHOLD - 1]
#             sequences.append(partial_sequence)
#         sequence_to_split_pattern_lookup.append( index + 
#             math.ceil(len(sequence) / LENGTH_THRESHOLD) - 1)

#     else:
#         # print(f"Index else: {index}")
#         sequences.append(c[index])
#         sequence_to_split_pattern_lookup.append(index)
#         # sequences.append(c1[index])
        
# print("-- Finished preparation --")
            
        
# # sequences = []
# # c1 = [c for _ in range (0,10)]

# # for s_index, sequence in tqdm(enumerate(c1)):
# # sequences.append([[event] if isinstance(
# #     event, int) else event for event in sequence])

# # if len(sequence) > PATTERN_LENGTH:
# #     for i in range(0, len(sequence), PATTERN_LENGTH):
# #         chopped_sequences.append([[event] if isinstance(
# #             event, int) else event for event in sequence[i:i + PATTERN_LENGTH]])
# # else:
# #     chopped_sequences.append([[event] if isinstance(
# #         event, int) else event for event in sequence])

# # print('chopped up')


# # print(chopped_sequences)
# patterns = get_frequent_patterns(sequences, max_gap=1)
# # patterns = get_frequent_patterns(c1, max_gap=1)
# print(f"\n#patterns found: {len(patterns)}")
# print("\n-- Done --")


### Remove patterns with length 0 or 1, sort on seq length

In [ ]:
# patterns = patterns[patterns['pattern'].apply(lambda x: len(x) > 2)]
patterns = patterns[patterns['pattern'].apply(lambda x: len(x) > 1)]
patterns['encoding'] = range(300, len(patterns) + 300)
patterns['aggregated'] = patterns.apply(lambda row: any(len(i.strip()) > 1 for i in row.pattern), axis=1)
patterns['seq_length'] = patterns.apply(lambda row: len(row.pattern), axis=1)
patterns = patterns.sort_values(by=['seq_length', 'sup'], ascending=False)
# patterns = patterns.sort_values(by=['aggregate'], ascending=True)
# patterns = patterns.sort_values(by=['seq_length', 'sup'], ascending=[False, True])
# patterns = patterns.sort_values(by=['sup', 'seq_length'], ascending=False)
# patterns = patterns.sort_values(by=['sup', 'seq_length'], ascending=[False, True])
# patterns = patterns.drop(columns=['seq_length'])
print(len(patterns))
patterns.head(n=20)


In [ ]:
# seq = [sequence['sequence'] for sequence in get_clustered_events(0)]
# seq = [sequence['sequence']
#        for sequence in get_clustered_events(cluster_level)]
# seq = sequence_to_list_of_strings([sequence['sequence']
#        for sequence in get_clustered_events(cluster_level)])
# seq = copy.deepcopy(c)
# seq = replace_gaps_to_string(c)
# seq[0]
# c[0]

### Insert patterns in sequences

In [ ]:
# seq = [sequence['sequence'] for sequence in get_clustered_events(cluster_level)].copy()
# seq = copy.deepcopy(c)
seq = replace_gaps_and_to_string(copy.deepcopy(c))

num_patterns_inserted = 0
inserted_patterns = []
# s_adj = []
# s_adj = ""
# printing = True

for index, sequence in tqdm(enumerate(seq)):
    seq_inserted = 0
    
    s = " " + sequence  + " "
    if s[0] != " ":
        s = s.ljust(len(s) + 1, " ")
    if s[-1] != " ":
        s = s.rjust(len(s) + 1, " ")
    # s_adj.append(s)
    # s_adj = copy.deepcopy(s)
    # print(s_adj)
    for pattern in patterns.itertuples():
        pat = " " + " ".join(pattern.pattern) + " "

        if pat[0] != " ":
            pat = pat.ljust(len(pat) + 1, " ")
        if pat[-1] != " ":
            pat = pat.rjust(len(pat) + 1, " ")
        
        # if printing:
        #     print(f"pat: -{pat}-")
        #     print(f"pat: {pattern.encoding}")
        #     printing = False

        s_old = s
        # print(s)
        s = s.replace(pat, str(pattern.encoding).center(
            len(str(pattern.encoding)) + 2, " "))

        if not s_old == s:
            seq[index] = s
            # print(f"pat: {pat}, {pattern.encoding}")
            # print(s[0:200])
            seq_inserted += 1
            num_patterns_inserted += 1
            inserted_patterns.append(pattern.encoding)
    
    # print(f"inserted in sequence {index}: {seq_inserted}")

print(f"number of patterns inserted: {num_patterns_inserted}")


### Format string to original representation

In [ ]:

for index, sequence in tqdm(enumerate(seq)):
    # print(type(sequence))
    if isinstance(sequence, str):
        seq[index] = sequence.strip().split(" ")
       
        
        for e_index, event in enumerate(seq[index]):
            if event.isdigit():
                if patterns.iloc[np.where(patterns.encoding.values == int(event))].aggregated.values[0]:
                    seq[index][e_index] = [int(event)]
                else:
                    seq[index][e_index] = int(event)
            else:
                if len(event) > 1:
                    seq[index][e_index] = [e for e in event]
                else:
                    seq[index][e_index] = event
    else: 
        print(f'[ERROR] detected other type: {type(sequence)}')    

In [ ]:
import csv

print("[INFO] Writing to file...")
#  Write to file
fileName = "sequences-" + str(100) + "-" + str(cluster_level) + ".csv"
with open(fileName, "w", newline='') as f:
    wr = csv.writer(f)
    wr.writerows(seq)

print("[INFO] Done.")


In [ ]:
patterns.to_csv(f'patterns-{number_of_stays}-level-{cluster_level}.csv')


In [ ]:
original_sequences = [seq['sequence']
                          for seq in get_clustered_events(cluster_level)]

for i in range(len(original_sequences)):
    len_o = len(original_sequences[i])
    len_n = len(seq[i])
    print(f"Original length: {len_o}")
    print(f"New length: {len_n}")
    print(f"Reduction: {(len_n - len_o)/ len_o * 100} ")
    print("----")
